In [61]:
import collections
import logging
from enum import Enum
from pathlib import Path
from socket import gethostname
from typing import Dict, NamedTuple, Sequence, Tuple

import dicomslide as ds
import highdicom as hd
import numpy as np
import pandas as pd
import scipy
from sklearn.mixture import GaussianMixture
from pydicom.dataset import Dataset
from pydicom.sr.coding import Code
from dicomweb_client import DICOMwebClient, DICOMClient
from dicomweb_client.log import configure_logging
from dicomweb_client.ext.gcp.session_utils import (
    create_session_from_gcp_credentials
)
from dicomweb_client.ext.gcp.uri import GoogleCloudHealthcareURL

In [36]:
logger = logging.getLogger('notebook')
configure_logging(0)
logger.setLevel(logging.INFO)

In [3]:
url = GoogleCloudHealthcareURL(
    project_id='idc-dev-etl',
    location='us-central1',
    dataset_id='idc',
    dicom_store_id='idc_v10_pathology'
)

In [4]:
client = DICOMwebClient(
    str(url),
    session=create_session_from_gcp_credentials()
)

In [5]:
COMPONENT_INVESTIGATED = Code('246094008', 'SCT', 'Component investigated')

class Colors(Enum):
    RED = (230, 43, 0)
    GREEN = (0, 225, 0)
    BLUE = (25, 84, 255)
    CYAN = (25, 225, 255)
    YELLOW = (255, 225, 0)
    MAGENTA = (255, 43, 255)
    WHITE = (225, 225, 225)

CHANNEL_GROUPS = (
    {
        'name': 'PD1-PDL1-CD45-Ecadherin',
        'settings': (
            (Code("19677004", "SCT", "CD45"), Colors.BLUE),
            (Code("C94697", "NCIt", "PD1"), Colors.GREEN),
            (Code("C96024", "NCIt", "PDL1"), Colors.RED),
            (Code("61792", "FMA", "E-Cadherin"), Colors.WHITE),
        ),
    },
    {
        'name': 'PanCK-Vimentin-CD45-Ki67',
        'settings': (
            (Code("259987000", "SCT", "Cytokeratin"), Colors.YELLOW),
            (Code("75925000", "SCT", "Vimentin"), Colors.MAGENTA),
            (Code("19677004", "SCT", "CD45"), Colors.CYAN),
            (Code("C0208804", "UMLS", "KI67"), Colors.WHITE),
        )
    },
    {
        'name': 'CD3-CD20-CD68',
        'settings': (
            (Code("24851008", "SCT", "DNA"), Colors.WHITE),
            (Code("44706009", "SCT", "CD3"), Colors.CYAN),
            (Code("82753007", "SCT", "CD20"), Colors.MAGENTA),
            (Code("31001006", "SCT", "CD68"), Colors.YELLOW),
        )
    },
)

In [17]:
def group_images_into_slides(
    client: DICOMClient
) -> Dict[Tuple[str, str], Tuple[str, str, str, Code]]:
    """Index slide microscopy images in a DICOM store.
    
    Parameters
    ----------
    client: dicomweb_client.DICOMclient
        DICOM client
        
    Returns
    -------
    Dict[Tuple[str, str], Tuple[str, str, str, Code]]    
        Mapping of Study Instance UID and Container Identifier
        to Series Instance UID, SOP Instance UID,
        Optical Path Identifier and the substance used for
        tissue staining
    
    """
    # Restrict to HTAN data
    found_studies = [
        Dataset.from_json(result)
        for result
        in client.search_for_studies(
            search_filters={'PatientName': 'HTA*'},
            get_remaining=True,
            fuzzymatching=True
        )
    ]
    lut = collections.defaultdict(list)
    for study in found_studies:
        found_sm_series = [
            Dataset.from_json(ds)
            for ds in client.search_for_series(
                study.StudyInstanceUID,
                search_filters={'Modality': 'SM'}
            )
        ]
        for series in found_sm_series:
            image_metadata = [
                Dataset.from_json(ds)
                for ds in client.retrieve_series_metadata(
                    study.StudyInstanceUID,
                    series.SeriesInstanceUID
                )
            ]
            for image in image_metadata:
                if image.SamplesPerPixel > 1:
                    continue
                key = (
                    image.StudyInstanceUID,
                    image.ContainerIdentifier,
                )
                preparation_steps = (
                    image
                        .SpecimenDescriptionSequence[0]
                        .SpecimenPreparationSequence
                )
                if len(preparation_steps) == 0:
                    stain = None
                else:
                    stains = []
                    staining_preparation_step_items = (
                        preparation_steps[-1]
                            .SpecimenPreparationStepContentItemSequence
                    )
                    for item in staining_preparation_step_items:
                        if item.ValueType != hd.sr.ValueTypeValues.CODE.value:
                            continue
                        name = hd.sr.utils.get_coded_name(item)
                        if name != COMPONENT_INVESTIGATED:
                            continue
                        value = hd.sr.utils.get_coded_value(item)
                        stains.append(value)
                    try:
                        stain = stains[0]
                    except IndexError:
                        stain = None
                value = (
                    image.SeriesInstanceUID,
                    image.SOPInstanceUID,
                    image.OpticalPathSequence[0].OpticalPathIdentifier,
                    stain
                )
                lut[key].append(value)
    return lut

In [7]:
class Channel(NamedTuple):

    images: Sequence[Dataset]
    voi_range: Tuple[int, int]
    color: Tuple[int, int, int]


def compute_limit_values(pixel_array: np.ndarray) -> Tuple[int, int]:
    """Compute minimum and maximum value of the value of interest (VOI) window.

    Parameters
    ----------
    pixel_array: numpy.ndarray
        Image region based on which limit values should be computed

    Returns
    -------
    min_value: int
        Mininum value (lower limit) of the VOI window
    max_value: int
        Maximum value (upper limit) of the VOI window

    """
    # Sample pixels with stride along each axis
    num_pixels_per_axis = 200
    row_stride = np.max([
        1,
        int(np.floor(pixel_array.shape[0] / num_pixels_per_axis))
    ])
    col_stride = np.max([
        1,
        int(np.floor(pixel_array.shape[1] / num_pixels_per_axis))
    ])
    row_indices = np.arange(0, pixel_array.shape[0], row_stride)
    col_indices = np.arange(0, pixel_array.shape[1], col_stride)
    # Slice one axis at a time to save memory
    rows = pixel_array[row_indices]
    values = rows[:, col_indices]

    # Log transform values
    log_values = np.log(values[values > 0])

    # Train model on log transformed pixel values
    model = GaussianMixture(3, max_iter=1000, tol=1e-6)
    model.fit(log_values.reshape((-1, 1)))

    # Get the parameters of the top two Gaussian distributions, which are
    # assumed to be tissue autoflorescence and stain fluorescence signal.
    # Ignore the Gaussian distribution with the lowest mean, which is assumed
    # to be camera background.
    means = model.means_[:, 0]
    _, index1, index2 = np.argsort(means)
    mean1, mean2 = means[[index1, index2]]
    std1, std2 = model.covariances_[[index1, index2], 0, 0] ** 0.5
    weights1, weights2 = model.weights_[[index1, index2]]

    # Choose upper limit value two standard deviations away from the mean of
    # the Gaussian that represents the stain fluorescence signal.
    lmax = mean2 + 2 * std2

    # Choose lower limit value in between the two Gaussians that represent the
    # tissue autoflorescence and stain fluorescence signal.
    x = np.linspace(mean1, mean2, 50)
    y1 = scipy.stats.norm(mean1, std1).pdf(x) * weights1
    y2 = scipy.stats.norm(mean2, std2).pdf(x) * weights2
    lmin = x[np.argmin(np.abs(y1 - y2))]
    if lmin >= mean2:
        lmin = mean2 - 2 * std2

    # Undo the log transformation to get the min and max value of interst (VOI)
    voi_min = np.exp(lmin)
    voi_max = np.exp(lmax)

    # Ensure that min and max values lie within the pixel values.
    voi_min = max(voi_min, values.min(), 0)
    voi_max = min(voi_max, values.max())

    return (voi_min, voi_max)


def _create_palette_color_lut_transformation(
    color: Tuple[int, int, int]
) -> hd.PaletteColorLUTTransformation:
    bits_per_entry = 16
    high_value = 255
    dtype = np.dtype(f'uint{bits_per_entry}')

    red_segment_values = [
        0, 1, 0,
        1, high_value, color[0],
    ]
    green_segment_values = [
        0, 1, 0,
        1, high_value, color[1],
    ]
    blue_segment_values = [
        0, 1, 0,
        1, high_value, color[2],
    ]

    transformation = hd.PaletteColorLUTTransformation(
        red_lut=hd.SegmentedPaletteColorLUT(
            first_mapped_value=0,
            segmented_lut_data=np.array(red_segment_values, dtype=dtype),
            color='red'
        ),
        green_lut=hd.SegmentedPaletteColorLUT(
            first_mapped_value=0,
            segmented_lut_data=np.array(green_segment_values, dtype=dtype),
            color='green'
        ),
        blue_lut=hd.SegmentedPaletteColorLUT(
            first_mapped_value=0,
            segmented_lut_data=np.array(blue_segment_values, dtype=dtype),
            color='blue'
        )
    )
    return transformation


def create_presentation_state(
    channels: Sequence[Channel],
    series_instance_uid: str,
    series_number: int,
    instance_number: int
) -> hd.pr.AdvancedBlendingPresentationState:
    """Create presentation state instance for data visualization.

    Parameters
    ----------
    channels: Sequence[Tuple[Sequence[pydicom.Dataset], Tuple[int, int], Tuple[int, int, int]]]
        Information about channels (optical paths) that should be visualized
    series_instance_uid: str
        DICOM Series Instance UID that should be assigned to created instance
    series_number: int
        Number of the series in the parent study
    instance_number: int
        Number of the created instance in the series

    Returns
    -------
    highdicom.pr.AdvancedBlendingPresentationState
        DICOM Advanced Blending Presentation State instance

    """  # noqa: E501
    referenced_images = []
    blending_configurations = []
    blending_display_inputs = []
    channel_descriptions = []
    for i, ch in enumerate(channels):
        voi_lut_transformation = hd.pr.SoftcopyVOILUTTransformation(
            window_center=float(np.mean(ch.voi_range)),
            window_width=float(np.abs(np.diff(ch.voi_range))),
        )
        palette_color_lut_transformation = \
            _create_palette_color_lut_transformation(ch.color)

        blending_input_number = i + 1  # one-based index
        configuration = hd.pr.AdvancedBlending(
            referenced_images=ch.images,
            blending_input_number=blending_input_number,
            voi_lut_transformations=[voi_lut_transformation],
            palette_color_lut_transformation=palette_color_lut_transformation
        )
        display_input = hd.pr.BlendingDisplayInput(blending_input_number)

        description = ch.images[0].OpticalPathSequence[0].OpticalPathDescription
        channel_descriptions.append(description)
        referenced_images.extend(ch.images)
        blending_configurations.append(configuration)
        blending_display_inputs.append(display_input)

    blending_display = hd.pr.BlendingDisplay(
        blending_mode=hd.pr.BlendingModeValues.EQUAL,
        blending_display_inputs=blending_display_inputs
    )

    return hd.pr.AdvancedBlendingPresentationState(
        referenced_images=referenced_images,
        blending=blending_configurations,
        blending_display=[blending_display],
        series_instance_uid=series_instance_uid,
        series_number=series_number,
        sop_instance_uid=hd.UID(),
        instance_number=instance_number,
        manufacturer='Herrmann Lab',
        manufacturer_model_name='timux',
        software_versions='0.1.0',
        device_serial_number=gethostname(),
        content_label='MULTIPLEXED_IF',
        content_description='-'.join(channel_descriptions),
        institution_name='Massachusetts General Hospital',
        institutional_department_name='Pathology',
        content_creator_name='Herrmann^Markus^D'
    )

In [18]:
slide_info = group_images_into_slides(client)

2022-07-02 15:12:08 | WARNING  | dicomweb_client.web                      | empty response


In [21]:
compound_mapping = collections.defaultdict(list)
for (study_instance_uid, container_id), values in slide_info.items():
    for v in values:
        compound = v[-1]
        if compound is not None:
            compound_mapping[compound].append((study_instance_uid, container_id))

In [22]:
for compound, identifiers in compound_mapping.items():
    print(f'Code("{compound.value}", "{compound.scheme_designator}", "{compound.meaning}")')
    print(len(set(identifiers)))
    print('--')

Code("24851008", "SCT", "DNA")
101
--
Code("146671000146103", "SCT", "Rat protein")
16
--
Code("C104064", "NCIt", "CD163")
109
--
Code("146321000146106", "SCT", "Rabbit protein")
16
--
Code("C104394", "NCIt", "FOXP3")
112
--
Code("C96024", "NCIt", "PDL1")
122
--
Code("61792", "FMA", "E-Cadherin")
101
--
Code("75925000", "SCT", "Vimentin")
101
--
Code("C25899", "NCIt", "CDX2")
16
--
Code("C17307", "NCIt", "Lamin")
16
--
Code("83475004", "SCT", "Desmin")
16
--
Code("4167003", "SCT", "CD31")
101
--
Code("146681000146101", "SCT", "Mouse protein")
16
--
Code("C17323", "NCIt", "PCNA")
23
--
Code("C0208804", "UMLS", "KI67")
116
--
Code("89048009", "SCT", "Collagen type IV")
20
--
Code("259987000", "SCT", "Cytokeratin")
100
--
Code("LP35599-7", "LN", "Smooth Muscle Actin")
101
--
Code("24655002", "SCT", "CD4")
118
--
Code("19677004", "SCT", "CD45")
142
--
Code("C94697", "NCIt", "PD1")
129
--
Code("82753007", "SCT", "CD20")
116
--
Code("31001006", "SCT", "CD68")
133
--
Code("C104109", "NCIt", "

In [50]:
presentation_states = []
for (study_instance_uid, container_id), values in slide_info.items():
    print(f'search for slides of study "{study_instance_uid}" and container "{container_id}"')
    try:
        matched_slides = ds.find_slides(client, study_instance_uid=study_instance_uid, container_id=container_id, pyramid_tolerance=1.0)
    except Exception as error:
        print(f'ERROR - skip slides of study "{study_instance_uid}" and container "{container_id}": {error}')
    print(f'found n={len(matched_slides)} slides')
    for slide in matched_slides:
        print(f'create presentation states for slide {slide}')
        series_instance_uid = hd.UID()
        series_number = 100
        instance_number = 1
        for group in CHANNEL_GROUPS:
            group_name = group['name']
            group_settings = group['settings']
            print(f'create presentation state "{group_name}"')
            channels = []
            for stain, color in group_settings:
                optical_path_identifiers = [
                    item[2]
                    for item in slide_info[(study_instance_uid, container_id)]
                    if item[3] is not None and item[3] == stain
                ]
                if len(optical_path_identifiers) == 0:
                    print(
                        f'ERROR - could not find an optical path for stain "{stain.meaning}" '
                        f'for slide "{container_id}" of study "{study_instance_uid}" '
                    )
                    continue
                channel_index = slide.get_channel_index(
                    channel_identifier=optical_path_identifiers[0],
                    channel_type='OPTICAL_PATH'
                )
                volume_images = slide.get_volume_images(
                    channel_index=channel_index
                )
                thumbnail_image = volume_images[-1]
                matrix = thumbnail_image.get_total_pixel_matrix(channel_index=0)
                pixel_array = np.squeeze(matrix[:, :, 0])
                voi_range = compute_limit_values(pixel_array)
                ch = Channel(
                    [image.metadata for image in volume_images],
                    (voi_range[0], voi_range[1] * 2.0),
                    color.value
                )
                channels.append(ch)
                
            if len(channels) < 2:
                print(
                    'WARNING - skip presentation state for '
                    f'slide "{container_id}" of study "{study_instance_uid}" '
                    'because of lack of matching optical paths'
                )
                continue
                
            pr_instance = create_presentation_state(
                channels,
                series_instance_uid,
                series_number,
                instance_number
            )
            instance_number += 1
            presentation_states.append(pr_instance)

search for slides of study "2.25.67204616595610313210404611407915801307" and container "HTA7_989_1000"
found n=1 slides
create presentation states for slide <Slide HTA7_989_1000 8e9ebac17f10f1a287a02f3ac01af3778fbe6e3e113a21d0b8e8e5eabdf50901>
create presentation state "PD1-PDL1-CD45-Ecadherin"
create presentation state "PanCK-Vimentin-CD45-Ki67"
create presentation state "CD3-CD20-CD68"
ERROR - could not find an optical path for stain "CD3" for slide "HTA7_989_1000" of study "2.25.67204616595610313210404611407915801307" 
search for slides of study "2.25.289188443974026654725116153923555879592" and container "HTA12_3_1108"
found n=1 slides
create presentation states for slide <Slide HTA12_3_1108 fa4e3c9570dc3eef1050fe30babf42289baecb42084631581d37311f62c7a757>
create presentation state "PD1-PDL1-CD45-Ecadherin"
create presentation state "PanCK-Vimentin-CD45-Ki67"
create presentation state "CD3-CD20-CD68"
search for slides of study "2.25.289188443974026654725116153923555879592" and cont

In [ ]:
# client.store_instances(presentation_states)

In [60]:
data_dir = Path('data')
data_inventory_rows = []
for state in presentation_states:
    container_id = state.ContainerIdentifier
    content_description = state.ContentDescription.replace(' ', '').replace('(2)', '')
    filename = f'{study_instance_uid}_{container_id}_{content_description}.dcm'
    filepath = data_dir.joinpath(filename)
    state.save_as(filepath)
    data_inventory_rows.append({
        'Filename': filename,
        'SOP Instance UID': state.SOPInstanceUID,
        'Patient ID': state.PatientID,
        'Clinical Trial Protocol ID': state.ClinicalTrialProtocolID,
        'Study Instance UID': state.StudyInstanceUID,
        'Series Instance UID': state.SeriesInstanceUID
    })
data_inventory_filepath = data_dir.joinpath('inventory.txt')
data_inventory = pd.DataFrame(data_inventory_rows)
data_inventory.to_csv(data_inventory_filepath, sep='\t')

In [57]:
len(set([state.ReferencedSeriesSequence[0].SeriesInstanceUID for state in presentation_states]))

105